# Getting Started with Hyperkite

This short introduction uses Hyperkite to:

1. Build a neural network that classifies images.

2. Train the model and optimize hyperparameters.

3. And finally evaluate the accuracy of the model.

This tutorial is part of the original [Hyperkite Documentation](https://hyperkite.ai/docs/getting-started/).

### Installation

Install the Hyperkite package by running the following command in your terminal:

In [1]:
pip install git+https://github.com/hyperkite/hyperkite.git

  Cloning https://github.com/hyperkite/hyperkite.git to /tmp/pip-req-build-fd_qhj5y
  Running command git clone -q https://github.com/hyperkite/hyperkite.git /tmp/pip-req-build-fd_qhj5y
  Created wheel for hyperkite: filename=hyperkite-0.1.2-py3-none-any.whl size=3846 sha256=96778ca6875660fedd1f785a51e5787e0c389432a5f1db511f005506ed165e0d
  Stored in directory: /tmp/pip-ephem-wheel-cache-m9kui02f/wheels/da/85/30/a8b2297d5c3422b5a7de91ce4baf21173ec0fca0d35b4f9cfd
Successfully built hyperkite
Note: you may need to restart the kernel to use updated packages.


### Define the hyperparameter space


Sign in to [Hyperkite](https://hyperkite.ai/), and press the 'Create Study' button.

<img src="https://github.com/hyperkite/Hyperbase/raw/master/hyperkite/docs/source/_static/screenshot_a.PNG" width="50%"></img>

A nice looking interface will allow you to define the hyperparameters you wish to optimize. In our case, let's call our study `tutorial_study`, and select a Uniform Range between `10` and `1000` named `n_layers`. We can use the default `Hyperopt` optimizer. Your screen should look like this:

<img src="https://github.com/hyperkite/Hyperbase/raw/master/hyperkite/docs/source/_static/screenshot_b.PNG" width="50%"></img>

After defining the hyperparameters you want to optimize in your Study, you will obtain an unique Study `key`. In the next part of the tutorial, this key will be used to easily link our Python training code with Hyperkite.

<img src="https://github.com/hyperkite/Hyperbase/raw/master/hyperkite/docs/source/_static/screenshot_c.PNG" width="50%"></img>

### Loading a Dataset
We obtain a collection of handwritten digits by loading the [Digit Dataset](https://scikit-learn.org/stable/auto_examples/datasets/plot_digits_last_image.html) from sklearn:

In [2]:
from sklearn import datasets

# Load dataset
digits = datasets.load_digits()

Split the dataset into a ```training```, ```validation``` and ```testing``` subsets.

| Split      | # Samples | Description |
|------------|-----------|-------------|
| Training   | 1400      | The largest portion of the data samples are used to directly train the model. |
| Validation | 100       | A small portion the data is withheld to evaluate different hyperparameters.
| Testing    | 297       | Another small portion of the data is witheld to evaluate the final performance. |

In [3]:
# Use the first 1400 digits for training
train_data = digits.data[:1400]
train_labels = digits.target[:1400]

# Use the next 150 digits for validation
val_data = digits.data[1400:1500]
val_labels = digits.target[1400:1500]

# Use the last (247) digits for testing
test_data = digits.data[1500:]
test_labels = digits.target[1500:]

print('Training size:', len(train_data))
print('Validation size:', len(val_data))
print('Testing size:', len(test_data))

Training size: 1400
Validation size: 100
Testing size: 297


In [4]:
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize=(16, 10))
for i in range(10):
    plt.subplot(1, 10, i+1)
    plt.axis('off')
    plt.title(i)
    plt.imshow(train_data[np.where(train_labels == i)[0][0]].reshape(8, 8), cmap='gray')
plt.savefig('dataset_example.png')
plt.show()

<Figure size 1600x1000 with 10 Axes>

### Training a Model

In this tutorial we will use the ```MLPClassifier``` neural network classifier from ```sklearn``` to fit our data. If you want, you can replace the model with a more advanced neural network using tools such as Keras, PyTorch or Tensorflow.

In [58]:
import hyperkite
study_key = '5e837a9666b14316d5c98a94'

In [ ]:
from tqdm import tqdm

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss

for _ in tqdm(range(10000)):
    # Retrive new hyperparameter values from Hyperkite
    trial = hyperkite.new_trial(study_key)

    values = trial.values
    
    # Initialize model using received hyper-parameter values
    algorithm = ['ball_tree', 'kd_tree', 'brute'][values['algorithm']]
    leaf_size = int(values['leaf_size'])
    n_neighbors = int(values['n_neighbors'])

    model = KNeighborsClassifier(algorithm=algorithm,
                                 leaf_size=leaf_size,
                                 n_neighbors=n_neighbors)
    
    # Train model
    model.fit(train_data, train_labels)
    
    # Report back validation loss
    val_loss = log_loss(val_labels, model.predict_proba(val_data))
    trial.report_loss(val_loss)

 67%|██████▋   | 6667/10000 [09:39<07:10,  7.74it/s]

### Final performance with Hyperkite

In [67]:
best_values = hyperkite.get_best_values(study_key)
print('Best values: ', best_values)

# Set-up best found settings
algorithm = ['ball_tree', 'kd_tree', 'brute'][values['algorithm']]
leaf_size = int(values['leaf_size'])
n_neighbors = int(values['n_neighbors'])

normal_model = KNeighborsClassifier()

tuned_model = KNeighborsClassifier(algorithm=algorithm,
                                   leaf_size=leaf_size,
                                   n_neighbors=n_neighbors)

# Train model
normal_model.fit(train_data, train_labels)
tuned_model.fit(train_data, train_labels)


# Evaluate final performance of normal model on test set
normal_predictions = normal_model.predict(test_data)
normal_accuracy = sum(normal_predictions == test_labels) / len(test_labels)
print('Final accuracy without Hyperkite:', normal_accuracy)

# Evaluate final performance of tuned model on test set
tuned_predictions = tuned_model.predict(test_data)
tuned_accuracy = sum(tuned_predictions == test_labels) / len(test_labels)
print('Final accuracy with Hyperkite:', tuned_accuracy)

Best values:  {'algorithm': 1, 'leaf_size': 513.0, 'n_neighbors': 5.0}
Final accuracy without Hyperkite: 0.9562289562289562
Final accuracy with Hyperkite: 0.936026936026936


Sweet! The model performs a lot better than without hyperparameter tuning. Also note that using our study `key` we can request the best parameter values from hyperkite at any time later.

## What's next?

Well, now that you have learned how to tune hyperparameters with Hyperkite you can start applying this skill on other machine learning models. Try to replace the MultiLayerPerceptron in the tutorial with other models from sklearn, or start training more advanced deep learning models using [TensorFlow](https://www.tensorflow.org/tutorials) and [PyTorch](https://pytorch.org/tutorials/) and try to optimize them with [Hyperkite](https://hyperkite.ai).